In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# One-Hot Embeddings: Bag-of-Words Classifier

In [2]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH"),
        ("Yo creo que si".split(), "SPANISH"),
        ("it is lost on me".split(), "ENGLISH")]

word2idx = {}
for sent, _ in data:
    for word in sent:
        if word not in word2idx:
            word2idx[word] = len(word2idx)
print(word2idx)

label2idx = {"SPANISH": 0, "ENGLISH": 1}

VOC_SIZE = len(word2idx)
N_LABEL = len(label2idx)

X = []
y = []
for sent, label in data:
    # It would create double by default
    vec = np.zeros(VOC_SIZE)
    for word in sent:
        vec[word2idx[word]] += 1
    X.append(vec)
    y.append(label2idx[label])

# X -> torch.float32
# y MUST BE torch.int64 (long tensor)
X = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y, dtype=torch.long)
print(X)
print(y)

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}
tensor([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1.,
         1., 1., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 1.

In [3]:
class BoWClassifier(nn.Module):
    def __init__(self, n_label, voc_size):
        # All units with differentiable parameters should be included in __init__ function
        # non-linearity function like tanh, ReLU and softmax without differentiable parameters
        # could not be included in __init__ function
        super(BoWClassifier, self).__init__()
        self.linear = nn.Linear(voc_size, n_label)
        
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim=-1)

In [4]:
model = BoWClassifier(N_LABEL, VOC_SIZE)

for param in model.parameters():
    print(param)

bow_vec = X[0]
log_probs = model(bow_vec)
print(log_probs)
print(log_probs.size())

Parameter containing:
tensor([[ 0.0423,  0.1634, -0.1956, -0.1778,  0.0160, -0.1017, -0.1704,  0.1913,
         -0.0800, -0.1414, -0.1784,  0.0867,  0.0386, -0.1598, -0.1630, -0.0549,
          0.0309,  0.0142,  0.0154,  0.0417, -0.0811,  0.0987, -0.1339, -0.1556,
         -0.1690,  0.0997],
        [ 0.0677, -0.1123, -0.0592,  0.0973,  0.0931,  0.0548, -0.0040, -0.1803,
         -0.0728, -0.1021,  0.0570,  0.1874, -0.1917,  0.0284,  0.0758, -0.0546,
          0.1711,  0.1806, -0.0040,  0.0564,  0.0246,  0.1423,  0.0391, -0.1918,
          0.1575, -0.1649]], requires_grad=True)
Parameter containing:
tensor([ 0.0009, -0.1950], requires_grad=True)
tensor([-0.7975, -0.5986], grad_fn=<LogSoftmaxBackward>)
torch.Size([2])


In [5]:
train_X, test_X = X[:4], X[4:]
train_y, test_y = y[:4], y[4:]

# Prediction result before training. 
for bow_vec, label in zip(test_X, test_y):
    log_probs = model(bow_vec)
    print(log_probs, label)

# The weight vector for specific word
# The weight vector works like an embedding vector, mapping a word (i.e., one-hot embedding vector) 
# to a dense vector, except for the bias. 
print(model.linear.weight[:, word2idx['creo']])

tensor([-0.9313, -0.5009], grad_fn=<LogSoftmaxBackward>) tensor(0)
tensor([-0.4294, -1.0523], grad_fn=<LogSoftmaxBackward>) tensor(1)
tensor([-0.1784,  0.0570], grad_fn=<SelectBackward>)


In [6]:
# Training
# Negative log-likelihood
loss_func = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    running_loss = 0.0
    for bow_vec, label in zip(train_X, train_y):
        # log_probs MUST BE a matrix to enter the loss function
        # use view-function to reshape!!
        log_probs = model(bow_vec.view(1, -1))
        
        # log_probs -> matrix
        # label -> vector
        loss = loss_func(log_probs, label.view(1))
        running_loss += loss.item()

        # Backward propagation
        model.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
            print(epoch + 1, running_loss)

10 0.4090864509344101
20 0.19626059383153915
30 0.12852158769965172
40 0.09542188234627247
50 0.07583856023848057
60 0.06290698004886508
70 0.05373444641008973
80 0.0468918401747942
90 0.04159188363701105
100 0.03736665518954396


In [7]:
# Prediction result after training. 
for bow_vec, label in zip(test_X, test_y):
    log_probs = model(bow_vec)
    print(log_probs, label)

# The weight vector for specific word
# The weight vector works like an embedding vector, mapping a word (i.e., one-hot embedding vector) 
# to a dense vector, except for the bias. 
print(model.linear.weight[:, word2idx['creo']])

tensor([-0.1854, -1.7763], grad_fn=<LogSoftmaxBackward>) tensor(0)
tensor([-2.2737, -0.1086], grad_fn=<LogSoftmaxBackward>) tensor(1)
tensor([ 0.2828, -0.4041], grad_fn=<SelectBackward>)


# Dense Embeddings
## `nn.Embedding`
Encode semantics in words. 

In [8]:
word2idx = {'hello': 0, 
            'world': 1, 
            'i': 2, 
            'am': 3, 
            'syuoni': 4}
# 5 vocabulary size, 4 embedding size. 
emb = nn.Embedding(5, 4)
emb.weight

Parameter containing:
tensor([[ 0.6536,  0.8697,  1.3789,  1.2900],
        [ 0.1049, -0.6755, -0.7243, -0.1683],
        [ 1.5976,  0.2021, -0.8438, -1.4043],
        [ 0.4315, -0.6146, -0.7818,  0.6703],
        [-0.0093, -0.1767,  0.8269,  0.2559]], requires_grad=True)

In [9]:
# Indexing-input MUST BE torch.long/torch.int64
t = torch.tensor(word2idx['hello'], dtype=torch.long)
t_emb = emb(t)
t_emb

tensor([0.6536, 0.8697, 1.3789, 1.2900], grad_fn=<EmbeddingBackward>)

In [10]:
# Input as a sequence
idx_seq = torch.tensor([word2idx[w] for w in "hello world".split()], dtype=torch.long)
embedded = emb(idx_seq)
embedded

tensor([[ 0.6536,  0.8697,  1.3789,  1.2900],
        [ 0.1049, -0.6755, -0.7243, -0.1683]], grad_fn=<EmbeddingBackward>)

In [11]:
# Input as a minibatch of sequences
idx_seq_batch = torch.tensor([[0, 1], 
                              [2, 3], 
                              [3, 4]], dtype=torch.long)
embedded = emb(idx_seq_batch)
embedded

tensor([[[ 0.6536,  0.8697,  1.3789,  1.2900],
         [ 0.1049, -0.6755, -0.7243, -0.1683]],

        [[ 1.5976,  0.2021, -0.8438, -1.4043],
         [ 0.4315, -0.6146, -0.7818,  0.6703]],

        [[ 0.4315, -0.6146, -0.7818,  0.6703],
         [-0.0093, -0.1767,  0.8269,  0.2559]]], grad_fn=<EmbeddingBackward>)

## `nn.EmbeddingBag`
Equivalent to `torch.nn.Embedding` followed by `torch.sum(dim=0)` / `torch.mean(dim=0)` / `torch.max(dim=0)`.

In [12]:
# 5 vocabulary size, 4 embedding size. 
emb_sum = nn.EmbeddingBag(5, 4, mode='sum')
emb_sum.weight

Parameter containing:
tensor([[ 1.6553, -1.2246,  0.5315,  0.5556],
        [-0.0711,  0.3570,  0.4204, -0.1150],
        [-0.4560, -1.3167, -0.4843,  1.6366],
        [-0.8941, -0.3663, -2.3747, -1.3274],
        [ 1.6401, -0.9622,  1.3261,  0.5586]], requires_grad=True)

In [13]:
# The input sequence is viewed as packed / concatenated from multiple individual sequences. 
# The offsets indicates the starting indexes of individual sequences. 
idx_seq = torch.tensor([word2idx[w] for w in "hello world i am syuoni".split()], dtype=torch.long)
offsets = torch.tensor([0, 2], dtype=torch.long)
embedded = emb_sum(idx_seq, offsets)
print(embedded)

print(emb_sum.weight[0:2].sum(dim=0))
print(emb_sum.weight[2:5].sum(dim=0))

tensor([[ 1.5842, -0.8676,  0.9518,  0.4405],
        [ 0.2901, -2.6451, -1.5330,  0.8678]], grad_fn=<EmbeddingBagBackward>)
tensor([ 1.5842, -0.8676,  0.9518,  0.4405], grad_fn=<SumBackward1>)
tensor([ 0.2901, -2.6451, -1.5330,  0.8678], grad_fn=<SumBackward1>)


In [14]:
# Input as a minibatch of sequences
idx_seq_batch = torch.tensor([[0, 1], 
                              [2, 3], 
                              [3, 4]], dtype=torch.long)
embedded = emb_sum(idx_seq_batch)
embedded

tensor([[ 1.5842, -0.8676,  0.9518,  0.4405],
        [-1.3501, -1.6830, -2.8590,  0.3093],
        [ 0.7461, -1.3285, -1.0487, -0.7688]], grad_fn=<EmbeddingBagBackward>)

# Training Embeddings: N-Gram Model 
In an n-gram language model, given a sequence of words $w$, we want to compute  
$$
P \left( w_{i} \left| w_{i-1}, w_{i-2}, ..., w_{i-n+1} \right. \right)  
$$  
where $w_{i}$ is the i-th word of the sequence.  

In [15]:
# Use TWO words to predict next word
CONTEXT_SIZE = 2
EMB_DIM = 10

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
print(trigrams[:3])

voc = set(test_sentence)
word2idx = {word: i for i, word in enumerate(voc)}
VOC_SIZE = len(voc)

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [16]:
class NGramModel(nn.Module):
    def __init__(self, voc_size, emb_dim, context_size):
        super(NGramModel, self).__init__()
        self.emb = nn.Embedding(voc_size, emb_dim)
        self.fc1 = nn.Linear(context_size*emb_dim, 128)
        self.fc2 = nn.Linear(128, voc_size)
        
    def forward(self, ins):
        # ins include several words (N=context_size)
        # self.emb(ins) -> (context_size, emb_dim)
        # self.emb(ins).view((1, -1)) -> (1, context_size*emb_dim)
        emb_ins = self.emb(ins).view((1, -1))
        outs = F.relu(self.fc1(emb_ins))
        outs = self.fc2(outs)
        log_probs = F.log_softmax(outs, dim=-1)
        return log_probs

In [17]:
loss_func = nn.NLLLoss()
model = NGramModel(VOC_SIZE, EMB_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:
        context_idxes = [word2idx[w] for w in context]
        context_var = torch.tensor(context_idxes, dtype=torch.long)
        target_var = torch.tensor([word2idx[target]], dtype=torch.long)
        
        model.zero_grad()
        log_probs = model(context_var)
        
        # log_probs -> matrix
        # target_var -> vector
        loss = loss_func(log_probs, target_var)        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(epoch, total_loss)

0 519.463969707489
1 516.9117701053619
2 514.376535654068
3 511.85786056518555
4 509.35493779182434
5 506.8663694858551
6 504.39074325561523
7 501.92601132392883
8 499.4740843772888
9 497.0334732532501


# Computing Word Embeddings: Continuous Bag-of-Words
In an continuous BOW model, given a sequence of words $w$, we want to compute  
$$
P \left( w_{i} \left| w_{i+n-1}, ..., w_{i+1}, w_{i-1}, ..., w_{i-n+1} \right. \right)  
$$  
where $w_{i}$ is the i-th word of the sequence.  

In [18]:
CONTEXT_SIZE = 2
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

voc = set(raw_text)
VOC_SIZE = len(voc)
word2idx = {word: i for i, word in enumerate(voc)}

data = [([raw_text[i-2], raw_text[i-1], raw_text[i+1], raw_text[i+2]], raw_text[i]) for i in range(2, len(raw_text)-2)]
print(data[:3])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study')]


In [19]:
class CBOW(nn.Module):
    def __init__(self, voc_size, emb_dim):
        super(CBOW, self).__init__()
        self.emb = nn.Embedding(voc_size, emb_dim)
        self.fc = nn.Linear(emb_dim, voc_size)
        
    def forward(self, ins):
        emb_ins = self.emb(ins)
        outs = self.fc(emb_ins.sum(dim=0, keepdim=True))
        log_probs = F.log_softmax(outs, dim=-1)
        return log_probs

In [20]:
loss_func = nn.NLLLoss()
model = CBOW(VOC_SIZE, EMB_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in data:
        context_idxes = [word2idx[w] for w in context]
        context_var = torch.tensor(context_idxes, dtype=torch.long)
        target_var = torch.tensor([word2idx[target]], dtype=torch.long)
        
        model.zero_grad()
        log_probs = model(context_var)
        loss = loss_func(log_probs, target_var)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print(epoch, total_loss)

0 270.51849126815796
1 267.3776705265045
2 264.2975478172302
3 261.275999546051
4 258.3110843896866
5 255.40103685855865
6 252.54424130916595
7 249.73921477794647
8 246.98459494113922
9 244.27910840511322
